# Evaluate the box below to initialize the web UI.

In [2]:
import binascii
import json
import os
import pandas as pd
import pandas.io.sql as psql
import qgrid 
import ray
import redis
import sys

from bokeh.plotting import figure, show
from bokeh.layouts import layout, widgetbox, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Div, CustomJS, Range1d
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, output_notebook, push_notebook

In [3]:
redis_address = os.environ["REDIS_ADDRESS"]
ray.init(redis_address=redis_address)

KeyError: 'REDIS_ADDRESS'

In [4]:
addr, port = ray.worker.global_worker.redis_address.split(":")
rc = redis.StrictRedis(host=addr, port=port)

AttributeError: 'Worker' object has no attribute 'redis_address'

# Task and Actor Information 

In [1]:
def task_profiles():
    """Fetch and return a list of task profiles.

    Returns:
      A tuple of two elements. The first element is a dictionary mapping the
        task ID of a task to a list of the profiling information for all of the
        executions of that task. The second element is a list of profiling
        information for tasks where the events have no task ID.
    """
    task_info = dict()
    event_names = rc.keys("event_log*")
    counter = 1
    for i in range(len(event_names)):
        event_list = rc.lrange(event_names[i], 0, -1)
        for event in event_list:
            event_dict = json.loads(event)
            task_id = ""
            for event in event_dict: 
                if "task_id" in event[3]: 
                    task_id = event[3]["task_id"]
            task_info[task_id] = dict()
            for event in event_dict:
                if event[1] == "ray:get_task" and event[2] == 1:
                    task_info[task_id]["get_task_start"] = event[0]
                if event[1] == "ray:get_task" and event[2] == 2:
                    task_info[task_id]["get_task_end"] = event[0]
                if event[1] == "ray:import_remote_function" and event[2] == 1:
                    task_info[task_id]["import_remote_start"] = event[0]
                if event[1] == "ray:import_remote_function" and event[2] == 2:
                    task_info[task_id]["import_remote_end"] = event[0]
                if event[1] == "ray:acquire_lock" and event[2] == 1:
                    task_info[task_id]["acquire_lock_start"] = event[0]
                if event[1] == "ray:acquire_lock" and event[2] == 2:
                    task_info[task_id]["acquire_lock_end"] = event[0]
                if event[1] == "ray:task:get_arguments" and event[2] == 1:
                    task_info[task_id]["get_arguments_start"] = event[0]
                if event[1] == "ray:task:get_arguments" and event[2] == 2:
                    task_info[task_id]["get_arguments_end"] = event[0]
                if event[1] == "ray:task:execute" and event[2] == 1:
                    task_info[task_id]["execute_start"] = event[0]
                if event[1] == "ray:task:execute" and event[2] == 2:
                    task_info[task_id]["execute_end"] = event[0]
                if event[1] == "ray:task:store_outputs" and event[2] == 1:
                    task_info[task_id]["store_outputs_start"] = event[0]
                if event[1] == "ray:task:store_outputs" and event[2] == 2:
                    task_info[task_id]["store_outputs_end"] = event[0]
                if "worker_id" in event[3]:
                    task_info[task_id]["worker_id"] = event[3]["worker_id"]
                if "function_name" in event[3]:
                    task_info[task_id]["function_name"] = event[3]["function_name"]
    return task_info

### Remote Function Information 


In [ ]:
fn_table = ray.global_state.function_table()
fn_list = []
if (len(fn_table) != 0): 
    for fn_id in fn_table:
        val = fn_table[fn_id]
        val["function_id"] = fn_id
        fn_list.append(val)
else: 
    fn_list.append({"DriverID": "None", "Module": "None", "Function": "None", "FunctionID": "None"})
qgrid.nbinstall(overwrite = True)
frame = pd.DataFrame(fn_list) 
frame.columns = ["DriverID", "Module", "Function", "FunctionID"]
qgrid.show_grid(frame)

### Task Information 

In [ ]:
from pandas.io.json import json_normalize

tt = ray.global_state.task_table()

tt_list = list(tt.values())
for d in tt_list:
    d['TaskSpec']['ReturnObjectIDs'] = [oid.hex() for oid in d['TaskSpec']['ReturnObjectIDs']]
    d['TaskSpec']['Args'] = [arg.hex() if isinstance(arg, ray.local_scheduler.ObjectID) else arg for arg in d['TaskSpec']['Args']]

task_df = json_normalize(tt_list)
task_df.columns = ["Local Scheduler ID", "State", "Actor Counter", "ActorID", "Arguments", "DriverID", "FunctionID", 
                  "Parent Counter", "Parent Task ID", "Required CPUs", "Required GPUs", "Return Object IDs", "TaskID" ]
qgrid.show_grid(task_df)

### Task Profiles 

In [ ]:
task_info = task_profiles()
task_breakdowns = dict()
if len(task_info) != 0: 
    for task_id, data in task_info.items(): 
        task_breakdowns[task_id] = dict()
        if "import_remote_start" and "import_remote_end" in data: 
            task_breakdowns[task_id]["import_remote_time"]  = data["import_remote_end"] - data["import_remote_start"] 
        task_breakdowns[task_id]["get_task_time"] = data["get_task_end"] - data["get_task_start"]
        task_breakdowns[task_id]["get_arguments"] = data["get_arguments_end"] - data["get_arguments_start"]
        task_breakdowns[task_id]["execute_time"] = data["execute_end"] - data["execute_start"]
        task_breakdowns[task_id]["store_outputs"] = data["store_outputs_end"] - data["store_outputs_start"]
        task_breakdowns[task_id]["acquire_lock"] = data["acquire_lock_end"] - data["acquire_lock_start"]
        task_breakdowns[task_id]["worker_id"] = data["worker_id"]
else: 
    task_breakdowns["None"] = {"Acquire Lock":"None", 
                               "Execute Time":"None", 
                               "Get Arguments":"None", 
                               "Get Task":"None", 
                               "Import Remote Function":"None", 
                               "Store Outputs":"None", 
                               "Worker ID":"None"}
df = pd.DataFrame.from_dict(task_breakdowns)
df_t = df.T
df_t.index.name = "Task ID"
df_t.columns = ["Acquire Lock", "Execute Time", "Get Arguments", "Get Task", "Import Remote Function", "Store Outputs", "Worker ID"] 
qgrid.show_grid(df_t)

### Task-Worker Placement Information 

In [ ]:
task_worker = dict()
for task_id, data in task_info.items(): 
    task_worker[task_id] = dict()
    task_worker[task_id]["worker_id"] = data["worker_id"]
    task_worker[task_id]["function_name"] = data["function_name"]
if len(task_worker) == 0: 
    task_worker["None"] = {"Function ID": "None", "Worker ID": "None"}
df = pd.DataFrame.from_dict(task_worker)
df_t = df.T 
df_t.index.name = "Task ID"
df_t.columns = ["Function Name", "Worker ID"]
qgrid.show_grid(df_t)

### Task Profile Visualization

In [2]:
task_profs = task_info 
print(task_profs)
workers = dict()

counter = 0 
for task_id, data in task_profs.items(): 
    worker_id = data["worker_id"]
    if worker_id not in workers: 
        workers[worker_id] = counter
        counter += 1
    data["wid"] = workers[worker_id]        

all_times = []
for data in task_info.values(): 
    all_times.append(data["acquire_lock_start"])
    all_times.append(data["acquire_lock_end"])
    all_times.append(data["get_arguments_start"])
    all_times.append(data["get_arguments_end"])
    all_times.append(data["execute_start"])
    all_times.append(data["execute_end"])
    all_times.append(data["store_outputs_start"])
    all_times.append(data["store_outputs_end"])

min_time = min(all_times)
for data in task_info.values(): 
    data["acquire_lock_start"] = data["acquire_lock_start"] - min_time 
    data["acquire_lock_end"] = data["acquire_lock_end"] - min_time
    data["execute_start"] = data["execute_start"] - min_time  
    data["execute_end"] = data["execute_end"] - min_time  
    data["get_arguments_start"] = data["get_arguments_start"] - min_time  
    data["get_arguments_end"] = data["get_arguments_end"] - min_time  
    data["store_outputs_start"] = data["store_outputs_start"] - min_time  
    data["store_outputs_end"] = data["store_outputs_end"] - min_time  

object_dict = {oid.hex(): v for oid, v in ray.global_state.object_table().items()}
for oid, data in object_dict.items(): 
    tid = data["TaskID"]
    if tid in task_info: 
        task_info[tid]["object_id"] = oid 
        
output_notebook()
source = ColumnDataSource(data=dict(
                                    x=[],
                                    y=[],
                                    worker_id=[],
                                    task_id=[],
                                    function_name=[],
                                    get_arguments_start=[],
                                    get_arguments_end=[],
                                    acquire_lock_start=[],
                                    acquire_lock_end=[],
                                    execute_start=[],
                                    execute_end=[],
                                    store_outputs_start=[],
                                    store_outputs_end=[],
                                    wid=[],
                                    oid=[]
                                    ))
axis_map = {
    "worker_id": "worker_id",
    "time": "time",
}
 
hover = HoverTool(tooltips=[
    ("TaskID", "@task_id"),
    ("Function Name", "@function_name"),
    ("WorkerID", "@wid"),
    ("ObjectID", "@oid"),
    ("Variables", "@vars")
])
 
x_axis = Select(title="Time in seconds", options=sorted(axis_map.keys()), value="time")
y_axis = Select(title="WorkerID", options=sorted(axis_map.keys()), value="worker_id")
 
p = figure(plot_height=600, plot_width=700, title="", toolbar_location="below", tools=[hover,"pan","wheel_zoom","box_zoom"], toolbar_sticky=False)
p.hbar(y="y", height=1, left="acquire_lock_start", right="acquire_lock_end", source=source, color="#FF8633", legend="Acquire lock")
p.hbar(y="y", height=1, left="get_arguments_start", right="get_arguments_end", source=source, color="#8033FF", legend="Get arguments")
p.hbar(y="y", height=1, left="execute_start", right="execute_end", source=source, color="#3390FF", legend="Execute task")
p.hbar(y="y", height=1, left="store_outputs_start", right="store_outputs_end", source=source, color="#33FF9C", legend="Store outputs")
 
def select(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val):
    selected_tasks = dict()
    for task_id, data in task_profs.items():
        if ((data["acquire_lock_end"]-data["acquire_lock_start"]) > lock_time):
            if ((data["get_arguments_end"]-data["get_arguments_start"]) > args_time):
                if ((data["execute_end"]-data["execute_start"]) > exec_time):
                    if ((data["store_outputs_end"]-data["store_outputs_start"]) > outputs_time):
                        # these checks aren't actually right, whoops
                        if task_id_val:
                            if task_id_val in data["task_id"]:
                                selected_tasks[task_id] = data
                        elif worker_id_val:
                            if worker_id_val in data["worker_id"]:
                                selected_tasks[task_id] = data
                        elif function_name_val:
                            if function_name_val in data["function_name"]:
                                selected_tasks[task_id] = data 
                        else:
                            selected_tasks[task_id] = data
    return selected_tasks
   
def update(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val):
    selected_tasks = select(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val)
    print('update: selected {}'.format(len(selected_tasks)))
    from collections import defaultdict
    df = defaultdict(list)
    for i, worker in selected_tasks.items():
        df['x'].append(worker["execute_start"])
        df['y'].append(worker["worker_id"])
        df['worker_id'].append(worker['worker_id'])
        df['task_id'].append(i)
        df['function_name'].append(worker['function_name'])
        df["get_arguments_start"].append(worker['get_arguments_start'])
        df["get_arguments_end"].append(worker['get_arguments_end'])
        df['acquire_lock_start'].append(worker['acquire_lock_start'])
        df['acquire_lock_end'].append(worker['acquire_lock_end'])
        df['store_outputs_start'].append(worker['store_outputs_start'])
        df['store_outputs_end'].append(worker['store_outputs_end'])
        df['execute_start'].append(worker['execute_start'])
        df['execute_end'].append(worker['execute_end'])
        df['wid'].append(worker["wid"])
        df['oid'].append(worker["object_id"])
 
    x_name = axis_map[x_axis.value]
    y_name = axis_map[y_axis.value]
    p.xaxis.axis_label = "Time in seconds"
    p.yaxis.axis_label = "Worker ID"
    p.title.text = "Task Information"
    source.data = dict(
        x= df["x"],
        y= df["y"],
        worker_id=df["worker_id"],
        task_id=df["task_id"],
        function_name=df["function_name"],
        acquire_lock_start=df['acquire_lock_start'],
        acquire_lock_end=df['acquire_lock_start'],
        get_arguments_start=df["get_arguments_start"],
        get_arguments_end=df["get_arguments_end"],
        store_outputs_start=df['store_outputs_start'],
        store_outputs_end=df['store_outputs_start'],
        execute_start=df['execute_start'],
        execute_end=df['execute_end'],
        wid=df["wid"],
        oid=df["oid"]
    )
    push_notebook()
    
show(p, notebook_handle=True)

from ipywidgets import interact
from ipywidgets import FloatSlider, Text

style = {'description_width': 'initial'}
lock_time = FloatSlider(value=0, min=0, max=10, step=.1, description='Time to acquire lock: ', style=style)
args_time = FloatSlider(value=0, min=0, max=10, step=.1, description='Time to get arguments: ', style=style)
exec_time = FloatSlider(value=0, min=0, max=10, step=.1, description='Time to execute task: ', style=style)
outputs_time = FloatSlider(value=0, min=0, max=10, step=.1, description='Time to store outputs: ', style=style)
worker_id_val = Text(value="", description='Worker ID:',disabled=False)
task_id_val = Text(value="", description='Task ID:',disabled=False)
function_name_val = Text(value="", description='Function name:',disabled=False)

interact(update, lock_time=lock_time , args_time=args_time, exec_time=exec_time, outputs_time=outputs_time, 
         worker_id_val=worker_id_val, task_id_val=task_id_val, function_name_val=function_name_val)

AttributeError: 'NoneType' object has no attribute 'keys'

### Actor Information 


In [ ]:
def hex_identifier(identifier):
    return binascii.hexlify(identifier).decode()

def binary_to_hex(identifier):
     hex_identifier = binascii.hexlify(identifier)

actor_info = dict()
actors = rc.keys("Actor*") 
if actors != []: 
    for actor in actors:
        actor_key_str = actor[len('Actor:'):]
        actor_info['Actor:{}'.format(hex_identifier(actor))] = rc.hgetall(actor)
        x = actor_info['Actor:{}'.format(hex_identifier(actor))]
        if b'class_id' in x: 
            class_key_bytes = x[b'class_id']
            x[b'class_id'] =format(hex_identifier(x[b'class_id']))
        if b'class'in x: 
            x[b'class'] = format(hex_identifier(x[b'class']))
        if b'driver_id' in x: 
            driver_id_bytes = x[b'driver_id']
            x[b'driver_id'] = format(hex_identifier(x[b'driver_id']))
        if b'class_name' in x: 
            class_name_bytes = x[b'class_name']
            x[b'class_name'] = format(hex_identifier(x[b'class_name']))
        if b'module' in x: 
            class_name_bytes = x[b'module']
            x[b'module'] = format(hex_identifier(x[b'module']))
        if b'actor_method_names' in x: 
            x[b'actor_method_names'] = format(hex_identifier(x[b'actor_method_names']))
else: 
    actor_info["None"] ={"Method":"None", "Class": "None", "ClassID": "None", "Class Name": "None", "DriverID": "None", "Module":"None", "Num GPUs":"None"}

actor_df = pd.DataFrame.from_dict(actor_info)
df = actor_df.T
df.index.name = "ActorID"
df.columns = [ "Method", "Class", "Class ID", "Class Name", "DriverID", "Module", "Num GPUs"] 
qgrid.show_grid(df)

# System State Information 


### Node Information

In [ ]:
ctable = ray.global_state.client_table()

client_list = []
for node_ip in ctable:
    for client in ctable[node_ip]:
        client["node_ip_address"] = node_ip
        client_list.append(client)

client_df = pd.DataFrame(client_list)
client_df.columns = ["Aux Address", "Client Type", "DB Client ID", "Deleted", "Local Scheduler Socket", "Num CPUs", "NumGPUs", "Node IP Address"]
qgrid.show_grid(client_df)

### Worker Information 

In [ ]:
workers = rc.keys("Worker*") 
worker_info = dict()
for worker in workers:
    worker_key_str = worker[len('Workers:'):]
    worker_key_bytes = worker_key_str.encode('latin-1')
    worker_info['Workers:{}'.format(hex_identifier(worker_key_bytes))] = rc.hgetall(worker)
table = pd.DataFrame.from_dict(worker_info)
table_t = table.T
table_t.index.name = "WorkerID"
table_t.columns = ["Local Scheduler Socket", "Node IP Address", "Plasma Manager Socket", "Plasma Store Socket", "Stderr File", "Stdout File"]
qgrid.show_grid(table_t)

### Object Store Information


In [ ]:
object_dict = {oid.hex(): v for oid, v in ray.global_state.object_table().items()}
if len(object_dict) == 0: 
    object_dict["None"] = {"Data Size":"None", "Hash":"None", "IsPut":"None", "ManagerIDs":"None", "TaskID":"None"}
object_df = pd.DataFrame(object_dict).transpose()
object_df.index.name = "ObjectID"
object_df.columns = ["Data Size", "Hash", "IsPut", "ManagerIDs", "TaskID"]
qgrid.show_grid(object_df)

# Error Information


In [ ]:
event_names = rc.keys("event_log*")
error_profiles = dict()
for i in range(len(event_names)):
    event_list = rc.lrange(event_names[i], 0, -1)
    for event in event_list:
        event_dict = json.loads(event)
        task_id = ""
        traceback = ""
        worker_id = ""
        start_time = -1
    for element in event_dict:
        if element[1] == "ray:task:execute" and element[2] == 1:
            start_time = element[0]
        if "task_id" in element[3] and "worker_id" in element[3]:
            task_id = element[3]["task_id"]
            worker_id = element[3]["worker_id"]
        if "traceback" in element[3]:
            traceback = element[3]["traceback"]
        if task_id != "" and worker_id != "" and traceback != "":
            if start_time != -1:
                error_profiles[task_id] = dict()
                error_profiles[task_id]["worker_id"] = worker_id
                error_profiles[task_id]["traceback"] = traceback
                error_profiles[task_id]["start_time"] = start_time
table = pd.DataFrame.from_dict(error_profiles) 
qgrid.show_grid(table.T)